## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [91]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self._codigo = codigo.zfill(4) if len(codigo) <= 4 else codigo
        self._nombre = nombre if 1 <= len(nombre) <= 100 else "Producto"
        self._precio = max(10, min(precio, 10000))
        self._tipo = tipo if 0 <= len(tipo) <= 20 else "General"
        self._cantidad = max(0, min(cantidad, 1000))

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 1000:
            self._cantidad = valor

    @property
    def tipo(self):
        return self._tipo

    def valorTotal(self):
        return self._precio * self._cantidad


In [92]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [94]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion if 1 <= len(descripcion) <= 100 else "Oferta"
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def aplicar(self, producto, cantidad):
        raise NotImplementedError("El método aplicar debe ser implementado en las subclases")

class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion="Descuento aplicado", codigos=[], tipos=[]):
        super().__init__(descripcion, codigos, tipos)
        self.descuento = max(0, min(descuento, 100))

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad) * (self.descuento / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, descripcion="Oferta 2x1", codigos=[], tipos=[]):
        super().__init__(descripcion, codigos, tipos)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            pares = cantidad // 2
            return pares * producto.precio
        return 0


In [95]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [96]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = {}
        self.ofertas = []

    @classmethod
    def leer(self, archivo):
        """Carga los productos desde un archivo CSV."""
        catalogo_temp = Catalogo()
        with open(archivo, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                producto = Producto(
                    row['codigo'],
                    row['nombre'],
                    int(row['precio']),
                    row['tipo'],
                    int(row['cantidad'])
                )
                catalogo_temp.agregar(producto)
        return catalogo_temp

    def guardar(self, archivo):
        """Guarda los productos en un archivo CSV."""
        with open(archivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos.values():
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    def agregar(self, *productos):
            """Agrega uno o más productos al catálogo."""
            for producto in productos:
                self.productos[producto.codigo] = producto

    def buscar(self, codigo):
        """Busca un producto por código."""
        return self.productos.get(codigo, None)

    def registrarOferta(self, oferta):
        """Registra una nueva oferta."""
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        """Busca la primera oferta aplicable a un producto y cantidad."""
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        """Calcula el descuento aplicable a un producto y cantidad."""
        oferta = self.buscarOferta(producto, cantidad)
        return oferta.calcularDescuento(producto, cantidad) if oferta else 0

    @property
    def cantidadProductos(self):
        """Cantidad de productos en el catálogo."""
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        """Cantidad total de unidades en el catálogo."""
        return sum(producto.cantidad for producto in self.productos.values())

    @property
    def valorTotal(self):
        """Valor total del catálogo sin descuentos."""
        return sum(producto.valorTotal() for producto in self.productos.values())

    def informe(self):
        """Genera un informe del catálogo con el formato especificado."""
        cantidad_productos = self.cantidadProductos
        cantidad_unidades = self.cantidadUnidades
        precio_promedio = self.valorTotal / cantidad_unidades if cantidad_unidades > 0 else 0
        valor_total = self.valorTotal

        informe = "INFORME CATALOGO\n"
        informe += f"Cantidad de productos:   {cantidad_productos}\n"
        informe += f"Cantidad de unidades:    {cantidad_unidades}\n"
        informe += f"Precio Promedio:       $ {precio_promedio:.2f}\n"
        informe += f"Valor total:           $ {valor_total:.2f}\n"

        tipos = {}
        for producto in self.productos.values():
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'total': 0}
            tipos[producto.tipo]['unidades'] += producto.cantidad
            tipos[producto.tipo]['total'] += producto.valorTotal()

        informe += "Tipos de productos: "
        for tipo, data in tipos.items():
            promedio = data['total'] / data['unidades'] if data['unidades'] > 0 else 0
            informe += f"  - {tipo:<20} :  {data['unidades']}u x $ {promedio:.2f}\n"

        informe += "Ofertas:\n"
        if not self.ofertas:
            informe += "  - No hay ofertas registradas\n"
        else:
            for oferta in self.ofertas:
                informe += f"  - {oferta.descripcion}\n"

        print("DEBUG: Informe generado:\n", informe)

        return informe

    def vender(self, producto, cantidad):
        """Actualiza la cantidad disponible de un producto tras una venta."""
        if producto.codigo in self.productos and cantidad <= self.productos[producto.codigo].cantidad:
            self.productos[producto.codigo].cantidad -= cantidad


In [97]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

DEBUG: Informe generado:
 INFORME CATALOGO
Cantidad de productos:   4
Cantidad de unidades:    97
Precio Promedio:       $ 1684.54
Valor total:           $ 163400.00
Tipos de productos:   - gaseosa              :  30u x $ 1300.00
  - galleta              :  67u x $ 1856.72
Ofertas:
  - Oferta 2x1
  - Descuento aplicado

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [99]:
class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if value == "":
            pass
        else:
            self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if self.validar_cuit(value):
            self._cuit = value

    def validar_cuit(self, cuit):
        """Valida el formato del CUIT."""
        return isinstance(cuit, str) and len(cuit) == 13 and cuit[2] == '-' and cuit[11] == '-' and cuit.replace('-', '').isdigit()


In [100]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [101]:
class Factura:
    _numero = 100

    @classmethod
    def ultimaFactura(cls, numero):
        """Establece el último número de factura."""
        cls._numero = numero

    @classmethod
    def nuevoNumero(cls):
        """Incrementa el número de factura y lo retorna."""
        cls._numero += 1
        return cls._numero

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.productos = {}
        self.numero = Factura.nuevoNumero()

    def agregar(self, producto, cantidad):
        """Agrega un producto y su cantidad a la factura."""
        if producto.codigo in self.productos:
            self.productos[producto.codigo] += cantidad
        else:
            self.productos[producto.codigo] = cantidad
        self.catalogo.vender(producto, cantidad)


    @property
    def cantidadProductos(self):
        """Cantidad de productos en la factura."""
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        """Cantidad total de unidades en la factura."""
        return sum(self.productos.values())

    @property
    def subtotal(self):
        """Calcula el subtotal de la factura."""
        total = 0
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            if producto:
                total += producto.precio * cantidad
        return total

    @property
    def descuentos(self):
        """Calcula el total de descuentos aplicados a la factura."""
        total_descuento = 0
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            if producto:
                total_descuento += self.catalogo.calcularDescuento(producto, cantidad)
        return total_descuento

    @property
    def total(self):
        """Calcula el total de la factura aplicando los descuentos."""
        return self.subtotal - self.descuentos

    def imprimir(self):
        """Genera una representación en texto de la factura."""
        impresion = f"Factura para: {self.cliente.nombre}\n"
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            if producto:
                impresion += f"{cantidad} {producto.nombre}\n"
    
        descuentos_aplicados = []
        for codigo, cantidad in self.productos.items():
            producto = self.catalogo.buscar(codigo)
            if producto:
                oferta = self.catalogo.buscarOferta(producto, cantidad)
                if oferta:
                    descuentos_aplicados.append(oferta.descripcion)

        if self.descuentos > 0:
            impresion += f"Descuento 10%\n"
        if descuentos_aplicados:
            for desc in descuentos_aplicados:
                impresion += f"{desc}\n"

        impresion += f"TOTAL: {self.total:.2f}"
        return impresion



In [102]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [103]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
